In [9]:
import os
import sys
import json
import pathlib

sys.path.append("/home/arnaik/OracleProject")

from src.datautils import read_jsonl

In [15]:
def load_linter_results(text):
    results = []
    idiom_code = None
    for line in text.split("\n"):
        line = line.strip()
        if line == "": continue
        elif line.startswith("**Idiom") and line.endswith("Violations:**"):
            idiom_code = line.removesuffix("Violations:**").removeprefix("**Idiom").strip()
        elif line == "NO VIOLATIONS FOUND": continue
        else:
            try: 
                result = json.loads(line)
                result["code"] = idiom_code
                results.append(result)
            except Exception as e: pass
                # print(e)
                # print(f"{e}: {line}")
    return results

In [11]:
cd /home/arnaik/OracleProject

/home/arnaik/OracleProject


In [23]:
for step in [0, 200, 400, 600, 800, 1000, 1200, 1240]:
    if step == 0:
        preds = read_jsonl("data/meta_linting_preds_vllm/qwen3_4b_sft_preds_4000_transfer_v5_lineno.jsonl")
    else: preds = read_jsonl(f"data/meta_linting_preds_vllm/qwen3_4b_dpo_preds_{step}_transfer_v5_lineno.jsonl")
    no_violation_acc = 0
    no_violation_tot = 0
    for rec in preds:
        # print(rec['ground_truth'])
        gt_results = load_linter_results(rec['ground_truth'])
        if len(gt_results) == 0:
            no_violation_tot += 1
            model_results = load_linter_results(rec['model_response'])
            if len(model_results) == 0:
                no_violation_acc += 1

    print(f"DPO {step} no-violation pred acc (%): {100*no_violation_acc/no_violation_tot:.2f}")

5300it [00:00, 52081.18it/s]


DPO 0 no-violation pred acc (%): 99.13


5300it [00:00, 328420.69it/s]


DPO 200 no-violation pred acc (%): 79.35


5300it [00:00, 226324.42it/s]


...

DPO 1000 no-violation pred acc (%): 75.81


5300it [00:00, 325506.44it/s]


DPO 1200 no-violation pred acc (%): 75.81


5300it [00:00, 327766.97it/s]


DPO 1240 no-violation pred acc (%): 75.45
